# Spike Locator

## Written by Michael George

Iterate through folders, looking for possible spikes in GPS files

In [1]:
import os
import sys

readersPath = os.path.join('.', 'readers')
if readersPath not in sys.path:
    sys.path.extend([readersPath])

from generic_reader import getReader

## Main Function

In [2]:
def findSpikes():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions', 'archive')

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext in ('.sbn', '.sbp'):
                filePath = os.path.join(root, file)
                reader = getReader(filePath)
                try:
                    reader.load()
                    
                    # Ignore GT-11 files
                    if not reader.header['firmware'].startswith('V1.62'):
                        maxSpeed = reader.data['sog'].max()
                        if maxSpeed > 25:
                            print('{} m/s in {}'.format(maxSpeed, filePath.replace(projdir + '/', '')))

                    #print('.', end='')
                except:
                    #print('E', end='')
                    #print('Failed to load {}'.format(filePath))
                    pass

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], ".."))

    findSpikes()
    
    print('All done!')

All done!
